<a href="https://colab.research.google.com/github/Khaled-Mohammed-Abdelgaber/deep-learning-projects-/blob/main/VGG_example_on_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries importing

In [2]:
import tensorflow as tf
from keras.layers import *
from keras.models import Model
import tensorflow_datasets as tfds

#Checking tensorflow version

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


#Building Block class
**vars()** its a built in funtion that enable us to access class attributs in form of string and create it also in form of string.
this thing will help us to use for loop to make repeated layers without need to excplicity declear it

In [4]:
class Block(Model):
  def __init__(self, filters, kernel_size, repetition, pool_size=2, strides=2,padding='same'):
    super(Block ,self).__init__()

    #attributes of class
    self.filters = filters
    self.repetition = repetition
    self.pool_size = pool_size
    self.kernel_size = kernel_size
    self.strides = strides
    self.padding = padding
    #convolution layers
    for i in range(self.repetition):
      vars(self)[f"Conv2D{i}"] = Conv2D(self.filters,self.kernel_size,padding=self.padding ,activation = 'relu')

    #maxpooling layer
    self.max_pooling = MaxPooling2D(pool_size = self.pool_size,strides=self.strides)
    

  def call(self , input_tensor):
    # access the class's conv2D_0 layer
    conv0 = vars(self)['Conv2D0']
    x = conv0(input_tensor)
    for i in range(1,self.repetition):
      x = vars(self)[f"Conv2D{i}"](x)
    max_pooling = self.max_pooling (x)

    return max_pooling



#Building VGG model:
this model will consist of multiple layers some of them will be **Block** class instance.
**Notes:**


*   if we want to classify multiple classes we will use **softmax** as activation function of last **Dense** layer and loss function will be **sparse_categorical_crossentropy** 
*   if we want to classify one class yes or no activation function of last layer should be **sigmoid** and loss function should be **binary_crossentropy** 



In [8]:
class VGG(Model):
  def __init__(self,num_classes):
    super(VGG,self).__init__()
    self.num_classes = num_classes
    self.block_a = Block(64,3,2)
    self.block_b = Block(128,3,3)
    self.block_c = Block(256,3,3)
    self.block_d = Block(512,3,3)
    self.block_e = Block(512,3,3)
    self.flatten = Flatten()
    self.fc = Dense(256,activation = 'relu')
    self.classifier = Dense(self.num_classes,activation = 'softmax')
  def call(self,input_tensor):
    x = self.block_a(input_tensor)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)
    return self.classifier(x)



#Datasets loading:
this function will be used to load datasets.


*   **tf.cast** work as casting of int() or float() but work with tensors. its inputs are
*   **map** work as well known map which apply function to datasets
*   **.batch()** change the shape


In [21]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']
dataset = dataset.map(preprocess).batch(32)


In [22]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [17]:
dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [14]:
model = VGG(2)
model.compile(optimizer = 'Adam' ,loss = 'sparse_categorical_crossentropy' ,metrics = ['accuracy'])

In [16]:
model.fit(dataset ,epochs = 5)

Epoch 1/5
727/727 [==============================] - 135s 185ms/step - loss: 0.6610 - accuracy: 0.6186
Epoch 2/5
727/727 [==============================] - 134s 184ms/step - loss: 0.6377 - accuracy: 0.6417
Epoch 3/5
727/727 [==============================] - 134s 184ms/step - loss: 0.6211 - accuracy: 0.6588
Epoch 4/5
727/727 [==============================] - 134s 184ms/step - loss: 0.6092 - accuracy: 0.6674
Epoch 5/5
727/727 [==============================] - 133s 183ms/step - loss: 0.6000 - accuracy: 0.6750
